In [ ]:
# @param data_dir temporal data storage for local execution
# @param file_name csv file name

In [ ]:
import pandas as pd
import os 

In [ ]:
file_name = os.environ.get('file_name', 'data.csv')
data_dir= os.environ.get('data_dir', '../../data/')

In [ ]:
df = pd.read_csv(data_dir+file_name)

In [ ]:
df['time'].min()


In [ ]:
df['time'].max()

In [2]:
!pip3 install pyspark==3.1.1

     |████████████████████████████████| 212.3 MB 4.7 kB/s eta 0:00:01     |████████████████████████▊       | 163.8 MB 4.1 MB/s eta 0:00:12��████████▊    | 183.9 MB 4.4 MB/s eta 0:00:07
     |████████████████████████████████| 198 kB 1.4 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=7f666eab8a29b1d2f797395a094dc20483167c13d20c15295ff782ee95c84044
  Stored in directory: /home/jovyan/.cache/pip/wheels/b3/0e/81/264aeed961e43b9f6ba9ec81c8c540d2d7dccc52c6b51cbf22
Successfully built pyspark


In [1]:
import pyspark

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
spark = SparkSession.builder.getOrCreate()

In [3]:
df = spark.read.option('header','true').csv('../../data/data.csv')

In [4]:
df.createOrReplaceTempView('df')

In [ ]:
spark.sql('select min(time), max(time) from df').show()

In [8]:
spark.sql("select * from df where sm <> 'null' and lat='74.625' and lon='87.875' order by time asc").show()

+----------+------+------+-----------+
|      time|   lat|   lon|         sm|
+----------+------+------+-----------+
|1979-06-01|74.625|87.875| 0.20680334|
|1979-07-01|74.625|87.875|  0.3784733|
|1979-08-01|74.625|87.875| 0.43579885|
|1979-09-01|74.625|87.875| 0.47746503|
|1980-06-01|74.625|87.875|       0.04|
|1980-07-01|74.625|87.875| 0.36691222|
|1980-08-01|74.625|87.875| 0.39871296|
|1980-09-01|74.625|87.875|   0.475062|
|1981-06-01|74.625|87.875| 0.19585133|
|1981-07-01|74.625|87.875| 0.33518302|
|1981-08-01|74.625|87.875| 0.38786596|
|1981-09-01|74.625|87.875| 0.41234347|
|1982-06-01|74.625|87.875| 0.26879317|
|1982-07-01|74.625|87.875| 0.37895218|
|1982-08-01|74.625|87.875|  0.3972472|
|1982-09-01|74.625|87.875| 0.45598736|
|1983-06-01|74.625|87.875|0.077149324|
|1983-07-01|74.625|87.875|  0.3582895|
|1983-08-01|74.625|87.875| 0.41817418|
|1983-09-01|74.625|87.875| 0.42353442|
+----------+------+------+-----------+
only showing top 20 rows



In [5]:
spark.sql('''

select count(*) from (
    select count(*) from df group by lon, lat
)
''').show()

+--------+
|count(1)|
+--------+
| 1036800|
+--------+



In [7]:
spark.sql('''
    select lon, lat from df where lat='74.625' and lon='87.875' group by lon, lat
''').rdd.map(lambda x: [x.lon, x.lat]).collect()

[['87.875', '74.625']]

In [9]:
sm = spark.sql("select sm from df where sm <> 'null' and lat='74.625' and lon='87.875' order by time asc").collect()

In [18]:
import numpy as np
sm = np.array(list(map(lambda x :x.sm, sm)))

In [23]:
sm = sm.astype(float)

In [27]:
coefficients, residuals, _, _, _ = np.polyfit(range(len(sm)),sm,1,full=True)
mse = residuals[0]/(len(sm))
nrmse = np.sqrt(mse)/(sm.max() - sm.min())
print('Slope ' + str(coefficients[0]))
print('NRMSE: ' + str(nrmse))

Slope 0.0004954139143477216
NRMSE: 0.15286518423836323
